In [1]:
import ee
from Py6S import *
import datetime
import math
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()),'bin'))
# from atmospheric import Atmospheric

import ee
import geemap

ee.Authenticate()
ee.Initialize()



Successfully saved authorization token.


In [2]:
startDate = ee.Date('2018-04-01')
endDate = ee.Date('2018-05-01')
Map = geemap.Map()
jharkhand= ee.FeatureCollection('projects/ee-tamsheelansari1/assets/jharkhand')
L8_col= ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA').filterBounds(jharkhand).filterDate(startDate,endDate).filterMetadata('CLOUD_COVER','less_than',20)
jh= L8_col.first()

Map.addLayer(jharkhand,{},'jharkhand')
Map.addLayer(jh,{'band':['B4','B3','B2'],'min':0,'max':0.25},'jh')
Map.centerObject(jharkhand,6)


Map

Map(center=[23.653915918218225, 85.5539359645643], controls=(WidgetControl(options=['position', 'transparent_b…

In [3]:
print(L8_col)
print(L8_col.size().getInfo())

ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "Collection.filter",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.filter",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.filter",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "ImageCollection.load",
                      "arguments": {
                        "id": {
                          "constantValue": "LANDSAT/LC08/C02/T1_TOA"
                        }
                      }
                    }
                  },
                  "filter": {
                    "functionInvocationValue": {
                      "functionName": "Filter.intersects",
                      "arguments": {
                        "leftField": {
            

In [4]:
toa = jh

In [5]:
class Atmospheric():

  def round_date(date,xhour):
    """
    rounds a date of to the closest 'x' hours
    """
    y = date.get('year')
    m = date.get('month')
    d = date.get('day')
    H = date.get('hour')
    HH = H.divide(xhour).round().multiply(xhour)
    return date.fromYMD(y,m,d).advance(HH,'hour')
  
  def round_month(date):
    """
    round date to closest month
    """
    # start of THIS month
    m1 = date.fromYMD(date.get('year'),date.get('month'),ee.Number(1))
    
    # start of NEXT month
    m2 = m1.advance(1,'month')
      
    # difference from date
    d1 = ee.Number(date.difference(m1,'day')).abs()
    d2 = ee.Number(date.difference(m2,'day')).abs()
    
    # return closest start of month
    return ee.Date(ee.Algorithms.If(d2.gt(d1),m1,m2))
  
  
  
  def water(jh,date):
    """
    Water vapour column above target at time of image aquisition.
    
    (Kalnay et al., 1996, The NCEP/NCAR 40-Year Reanalysis Project. Bull. 
    Amer. Meteor. Soc., 77, 437-471)
    """
    
    # Point geometry required
    centroid = jharkhand
    
    # H2O datetime is in 6 hour intervals
    H2O_date = Atmospheric.round_date(date,6)
    
    # filtered water collection
    water_ic = ee.ImageCollection('NCEP_RE/surface_wv').filterDate(H2O_date, H2O_date.advance(1,'month'))
    
    # water image
    water_img = ee.Image(water_ic.first())
    
    # water_vapour at target
    water = water_img.reduceRegion(reducer=ee.Reducer.mean(), geometry=centroid).get('pr_wtr')
                                        
    # convert to Py6S units (Google = kg/m^2, Py6S = g/cm^2)
    water_Py6S_units = ee.Number(water).divide(10)                                   
    
    return water_Py6S_units
  
  
  
  def ozone(jh,date):
    """
    returns ozone measurement from merged TOMS/OMI dataset
    
    OR
    
    uses our fill value (which is mean value for that latlon and day-of-year)
  
    """
    
    # Point geometry required
    centroid = jharkhand
       
    def ozone_measurement(centroid,O3_date):
      
      # filtered ozone collection
      ozone_ic = ee.ImageCollection('TOMS/MERGED').filterDate(O3_date, O3_date.advance(1,'month'))
      
      # ozone image
      ozone_img = ee.Image(ozone_ic.first())
      
      # ozone value IF TOMS/OMI image exists ELSE use fill value
      ozone = ee.Algorithms.If(ozone_img,\
      ozone_img.reduceRegion(reducer=ee.Reducer.mean(), geometry=centroid).get('ozone'),\
      ozone_fill(centroid,O3_date))
      
      return ozone
      
    def ozone_fill(centroid,O3_date):
      """
      Gets our ozone fill value (i.e. mean value for that doy and latlon)
      
      you can see it
      1) compared to LEDAPS: https://code.earthengine.google.com/8e62a5a66e4920e701813e43c0ecb83e
      2) as a video: https://www.youtube.com/watch?v=rgqwvMRVguI&feature=youtu.be
      
      """
      
      # ozone fills (i.e. one band per doy)
      ozone_fills = ee.ImageCollection('users/samsammurphy/public/ozone_fill').toList(366)
      
      # day of year index
      jan01 = ee.Date.fromYMD(O3_date.get('year'),1,1)
      doy_index = date.difference(jan01,'day').toInt()# (NB. index is one less than doy, so no need to +1)
      
      # day of year image
      fill_image = ee.Image(ozone_fills.get(doy_index))
      
      # return scalar fill value
      return fill_image.reduceRegion(reducer=ee.Reducer.mean(), geometry=centroid).get('ozone')
     
    # O3 datetime in 24 hour intervals
    O3_date = Atmospheric.round_date(date,24)
    
    # TOMS temporal gap
    TOMS_gap = ee.DateRange('1994-11-01','1996-08-01')  
    
    # avoid TOMS gap entirely
    ozone = ee.Algorithms.If(TOMS_gap.contains(O3_date),ozone_fill(centroid,O3_date),ozone_measurement(centroid,O3_date))
    
    # fix other data gaps (e.g. spatial, missing images, etc..)
    ozone = ee.Algorithms.If(ozone,ozone,ozone_fill(centroid,O3_date))
    
    #convert to Py6S units 
    ozone_Py6S_units = ee.Number(ozone).divide(1000)# (i.e. Dobson units are milli-atm-cm )                             
    
    return ozone_Py6S_units
 

  def aerosol(geom,date):
    """
    Aerosol Optical Thickness.
    
    try:
      MODIS Aerosol Product (monthly)
    except:
      fill value
    """
    
    def aerosol_fill(date):
      """
      MODIS AOT fill value for this month (i.e. no data gaps)
      """
      return ee.Image('users/samsammurphy/public/AOT_stack')\
               .select([ee.String('AOT_').cat(date.format('M'))])\
               .rename(['AOT_550'])
               
               
    def aerosol_this_month(date):
      """
      MODIS AOT original data product for this month (i.e. some data gaps)
      """
      # image for this month
      img =  ee.Image(\
                      ee.ImageCollection('MODIS/006/MOD08_M3')\
                        .filterDate(Atmospheric.round_month(date))\
                        .first()\
                     )
      
      # fill missing month (?)
      img = ee.Algorithms.If(img,\
                               # all good
                               img\
                               .select(['Aerosol_Optical_Depth_Land_Mean_Mean_550'])\
                               .divide(1000)\
                               .rename(['AOT_550']),\
                              # missing month
                                aerosol_fill(date))
                      
      return img    
        
  
    def get_AOT(AOT_band,geom):
      """
      AOT scalar value for target
      """  
      return ee.Image(AOT_band).reduceRegion(reducer=ee.Reducer.mean(),\
                                 geometry=jharkhand)\
                                .get('AOT_550')
                                

    after_modis_start = date.difference(ee.Date('2000-03-01'),'month').gt(0)
    
    AOT_band = ee.Algorithms.If(after_modis_start, aerosol_this_month(date), aerosol_fill(date))
    
    AOT = get_AOT(AOT_band,geom)
    
    AOT = ee.Algorithms.If(AOT,AOT,get_AOT(aerosol_fill(date),geom))
    # i.e. check reduce region worked (else force fill value)
    
    return AOT

In [47]:
def func1(img):
    img = ee.Image(img)
    info = img.getInfo()['properties']
    scene_date = datetime.datetime.utcfromtimestamp(info['system:time_start']/1000)
    solar_elv = img.getInfo()['properties']['SUN_ELEVATION']
    solar_z = ee.Number(90).subtract(solar_elv).getInfo()
    h2o = Atmospheric.water(jh,img.date()).getInfo()
    o3 = Atmospheric.ozone(jh,img.date()).getInfo()
    aot = Atmospheric.aerosol(jh,img.date()).getInfo()
    SRTM = ee.Image('CGIAR/SRTM90_V4')# Shuttle Radar Topography mission covers *most* of the Earth
    alt = SRTM.reduceRegion(reducer = ee.Reducer.mean(),geometry = jharkhand, scale = 100).get('elevation').getInfo()
    km = alt/1000 # i.e. Py6S uses units of kilometers
    # Instantiate
    s = SixS()
    # Atmospheric constituents
    s.atmos_profile = AtmosProfile.UserWaterAndOzone(h2o,o3)
    s.aero_profile = AeroProfile.Continental
    s.aot550 = aot
    # Earth-Sun-satellite geometry
    s.geometry = Geometry.User()
    s.geometry.view_z = 0               # always NADIR (I think..)
    #s.geometry.solar_z = solar_z        # solar zenith angle
    s.geometry.solar_z = solar_z # solar zenith angle
    s.geometry.month = scene_date.month # month and day used for Earth-Sun distance
    s.geometry.day = scene_date.day     # month and day used for Earth-Sun distance
    s.altitudes.set_sensor_satellite_level()
    s.altitudes.set_target_custom_altitude(km)
    
    def spectralResponseFunction(bandname):        
        bandSelect = {
            'B1':PredefinedWavelengths.LANDSAT_OLI_B1,
            'B2':PredefinedWavelengths.LANDSAT_OLI_B2,
            'B3':PredefinedWavelengths.LANDSAT_OLI_B3,
            'B4':PredefinedWavelengths.LANDSAT_OLI_B4,
            'B5':PredefinedWavelengths.LANDSAT_OLI_B5,
            'B6':PredefinedWavelengths.LANDSAT_OLI_B6,
            'B7':PredefinedWavelengths.LANDSAT_OLI_B7,
            'B8':PredefinedWavelengths.LANDSAT_OLI_B8,
            'B9':PredefinedWavelengths.LANDSAT_OLI_B9,
                    }
        return Wavelength(bandSelect[bandname])
    def toa_to_rad(bandname):
        ESUN_L8 = [1895.33, 2004.57, 1820.75, 1549.49, 951.76, 247.55, 85.46, 1723.8, 366.97]
        ESUN_BAND = {
            'B1':ESUN_L8[0],
            'B2':ESUN_L8[1],
            'B3':ESUN_L8[2],
            'B4':ESUN_L8[3],
            'B5':ESUN_L8[4],
            'B6':ESUN_L8[5],
            'B7':ESUN_L8[6],
            'B8':ESUN_L8[7],
            'B9':ESUN_L8[8],
            }
        solar_angle_correction = math.cos(math.radians(solar_z))
        # Earth-Sun distance (from day of year)
        doy = scene_date.timetuple().tm_yday
        d = 1 - 0.01672 * math.cos(0.9856 * (doy-4))# http://physics.stackexchange.com/questions/177949/earth-sun-distance-on-a-given-day-of-the-year
        # conversion factor
        multiplier = ESUN_BAND[bandname]*solar_angle_correction/(math.pi*d**2)
        # at-sensor radiance
        rad = jh.select(bandname).multiply(multiplier)
        return rad
    def surface_reflectance(bandname):
        # run 6S for this waveband
        s.wavelength = spectralResponseFunction(bandname)
        s.run()

        # extract 6S outputs
        Edir = s.outputs.direct_solar_irradiance             #direct solar irradiance
        Edif = s.outputs.diffuse_solar_irradiance            #diffuse solar irradiance
        Lp   = s.outputs.atmospheric_intrinsic_radiance      #path radiance
        absorb  = s.outputs.trans['global_gas'].upward       #absorption transmissivity
        scatter = s.outputs.trans['total_scattering'].upward #scattering transmissivity
        tau2 = absorb*scatter                                #total transmissivity

        # radiance to surface reflectance
        rad = toa_to_rad(bandname)
        ref = rad.subtract(Lp).multiply(math.pi).divide(tau2*(Edir+Edif))
        return ref
    
    ca  = surface_reflectance('B1')
    blue = surface_reflectance('B2')
    green = surface_reflectance('B3')
    red = surface_reflectance('B4')
    nir = surface_reflectance('B5')
    swir1 = surface_reflectance('B6')
    swir2 = surface_reflectance('B7')
    
    ref = jh.select('BQA').addBands(ca).addBands(blue).addBands(green).addBands(red).addBands(nir).addBands(swir1).addBands(swir2)
    
    # dateString = scene_date.strftime ("%Y-%m-%d")
    # ref = ref.copyProperties(img).set({              
    #             #   'AC_date':dateString,
    #               'AC_aerosol_optical_thickness':aot,
    #               'AC_water_vapour':h2o,
    #               'AC_version':'py6S',
    #               'AC_contact':'tamsheelansari1@gmail.com',
    #               'AC_ozone':o3})
    
    return func1    

    # # define YOUR assetID 
    # # in my case it was something like this..
    # #assetID = 'users/samsammurphy/shared/sentinel2/6S/ESRIN_'+dateString
    # #assetID = 'users/ndminhhus/eLEAF/nt/s2_SIAC/'+fname,
    # # # export
    # # fname = ee.String(jh.get('system:index')).getInfo()
    # # export = ee.batch.Export.image.toAsset(\
    # #     image=ref,
    # #     description= 'L8_BOA_'+fname,
    # #     assetId = 'users/ndminhhus/eLEAF/nt/L8_py6S/'+'L8_BOA'+fname,
    # #     region = region,
    # #     scale = 30,
    # #     maxPixels = 1e13)

    # # # uncomment to run the export
    # # export.start()
    # # print('exporting ' +fname + '--->done')

In [48]:
boa = ee.Image(func1(toa)) 

EEException: Unrecognized argument type to convert to an Image: <function func1 at 0x000001BC38079DC0>

In [44]:
print(boa.getInfo())

{'type': 'Image', 'bands': [{'id': 'constant', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 0}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [18]:
Map= geemap.Map()
# Map = geemap.Map(center=[40,-100], zoom=4)
Map.addLayer(boa,{'band':['B4','B3','B2'],'min':0,'max':0.30},'boa')
Map.centerObject(jharkhand,8)
Map

Map(center=[23.653915918218225, 85.5539359645643], controls=(WidgetControl(options=['position', 'transparent_b…

In [124]:
print(boa.getInfo()['properties']['AC_date'])

KeyError: 'AC_date'